In [14]:
import pandas as pd
import re
import numpy as np
from collections import defaultdict

path=r'../datas/Option_A/Argentina_Supply and demand series_Time_by_rows.xlsx'
data=pd.read_excel(path)#,skiprows=range(3)


# Find the index of rows that match a specific expression using regular expressions
regex_pattern = r'^Argentina.*'  # Example regex pattern to match Argentina
indexes = data[data['Unnamed: 0'].str.contains(regex_pattern, regex=True, na=False)].index
indexes=list(indexes)
indexes.append(indexes[-1]+56)
print(indexes)

# Find the List of Items
regex_pattern = r'^Argentina - (.*)'  # Example regex pattern to match Argentina
# Extract the desired part using regular expressions
extracted_list = data.loc[data['Unnamed: 0'].str.contains(regex_pattern, regex=True, na=False), 'Unnamed: 0'].str.extract(regex_pattern, expand=False).to_list()
#print(extracted_list)

Dict=dict()

for i in range(len(extracted_list)):
    Dict[extracted_list[i]]=data[indexes[i]:indexes[i+1]]
    Dict[extracted_list[i]]=Dict[extracted_list[i]][1:]
    new_header = Dict[extracted_list[i]].iloc[0] #grab the first row for the header
    Dict[extracted_list[i]] = Dict[extracted_list[i]][1:] #take the data less the header row
    Dict[extracted_list[i]].columns = new_header #set the header row as the df header 
    Dict[extracted_list[i]] = Dict[extracted_list[i]].reset_index(drop=True)
    Dict[extracted_list[i]]=Dict[extracted_list[i]].iloc[1:]
    Dict[extracted_list[i]]=Dict[extracted_list[i]].iloc[:-1]
    Dict[extracted_list[i]]=Dict[extracted_list[i]].rename(columns={np.nan: 'Year'})
    # Round the values of each column to 2 decimal places
    Dict[extracted_list[i]] = Dict[extracted_list[i]].applymap(lambda x: round(x, 2) if pd.notnull(x) and isinstance(x, (int, float)) else x)


    
Dict[extracted_list[0]]

[2, 58, 114, 170, 226, 282, 338, 394, 450, 506, 562, 618, 674, 730, 786, 842, 898, 954, 1010, 1066, 1122, 1178, 1234, 1290, 1346, 1402, 1458]


/var/folders/cr/7sx_pjr5131byrt21svnsg0w0000gn/T/ipykernel_56733/111135797.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  extracted_list = data.loc[data['Unnamed: 0'].str.contains(regex_pattern, regex=True, na=False), 'Unnamed: 0'].str.extract(regex_pattern, expand=False).to_list()


3,Year,Oil,Natural gas,Coal,Hydroenergy,Nuclear,Firewood,Sugarcane and products,Other Primary_x000D_\n,Total Primaries,...,Kerosene/jet fuel,Diesel oil,Fuel oil,Coke,Charcoal,Gases,Other secondary,Non-energy,Total Secundaries,Total
1,1970,201770.00,67776.79,3628.47,1670.00,NaN,10230.40,3459.99,8670.00,297205.66,...,12200.52,48290.00,87469.99,999.59,3392.95,7970.14,5040.00,9400.00,241923.17,297205.66
2,1971,217399.69,72366.62,3728.77,1660.00,NaN,9630.10,3270.00,8580.00,316635.19,...,11384.07,57611.40,92785.95,1101.59,3529.45,8476.81,5430.00,10240.01,263032.38,316635.19
3,1972,222960.51,71769.03,3988.37,1620.02,NaN,9068.90,4280.00,6839.99,320526.82,...,11418.47,57550.50,92239.54,550.80,2417.96,8335.70,5380.00,11180.01,265324.63,320526.82
4,1973,216299.35,76566.36,2660.88,3220.25,NaN,6929.90,5940.01,5880.00,317496.74,...,11700.33,59464.50,87609.05,1699.99,2001.97,8278.23,5890.00,12340.00,272421.32,317496.74
5,1974,212600.10,74059.79,3687.47,5370.00,4540.39,9988.90,5470.01,6149.99,321866.67,...,13133.43,57959.39,84608.63,958.79,3477.45,9478.67,5659.45,14820.00,272776.19,321866.67
6,1975,203269.69,88061.69,2961.78,5590.09,8770.18,5920.20,5330.00,5700.00,325603.61,...,12815.84,54958.39,80996.69,1019.99,1930.47,8473.55,5050.00,14899.99,258811.56,325603.61
7,1976,204859.70,97838.94,3628.47,5390.09,9050.12,5710.01,5310.00,5150.00,336937.32,...,12991.84,60351.89,84071.35,822.79,2001.97,9330.19,5180.00,15240.00,271358.13,336937.32
8,1977,221670.00,98967.72,3150.58,6200.00,6528.67,6089.99,5700.00,5919.99,354226.94,...,13081.82,64675.79,90609.63,1509.59,2112.47,9564.30,5670.00,16539.99,289640.58,354226.94
9,1978,232359.99,98951.12,2560.58,8330.00,10683.27,7619.90,5270.00,6110.00,371884.87,...,12684.84,65693.70,87692.39,149.60,3236.95,10058.77,6810.00,15390.01,290340.12,371884.87
10,1979,242789.99,111641.63,4289.27,11460.00,9629.78,5549.90,5369.99,6379.99,397110.56,...,11826.04,69365.75,88599.80,591.59,2138.47,8438.85,6149.99,17540.00,298703.85,397110.56


In [4]:
for k in Dict.keys():
    print(k)
    df_k=Dict[k]
    #print(df_k)
    

Production
Import
Export
Stock change
Unused
Total supply
Refineries
Power plants 
Self-producers
Gas plants
Charcoal plants
Coke plants and blast furnaces_x000D_
Distilleries
Other centers
Total transformation
Own consumption
Losses
Adjustment
Transport 
Industrial
Residential
Commercial, services, public
Agriculture, fishing and mining
Energy consumption
Non-energy consumption
Final consumption


In [13]:
with pd.ExcelWriter("output.xlsx") as writer:
    for k in Dict.keys():
        Dict[k].to_excel(writer, sheet_name=k, index=False)
    

InvalidWorksheetName: Excel worksheet name 'Coke plants and blast furnaces_x000D_' must be <= 31 chars.